Restarted aspire (Python 3.8.18)

In [1]:
import aspire
import numpy as np
import pandas as pd
from aspire.operators import RadialCTFFilter
from aspire.source.simulation import Simulation
from aspire.volume import LegacyVolume, Volume
from utils import volsCovarEigenvec
import time
from covar_estimation import im_stack_backward
import torch
# Specify parameters
img_size = 15  # image size in square
num_imgs = 2000  # number of images
dtype = np.float32


rank = 4
c = rank + 1
vols = LegacyVolume(
    L=img_size,
    C=c,
    dtype=dtype,
).generate()
vols -= np.mean(vols,axis=0)
sim = Simulation(
    #unique_filters=[RadialCTFFilter(defocus=d) for d in np.linspace(1.5e4, 2.5e4, 7)],
    n=num_imgs,
    vols=vols,
    dtype=dtype,
    amplitudes=1,
    offsets = 0
)

vectorsGD = torch.tensor(volsCovarEigenvec(vols).asnumpy(),requires_grad = False)

2024-03-24 14:24:59,956 INFO [aspire.source.image] Creating Simulation with 2000 images.


In [7]:
%load_ext autoreload
%autoreload 2

from covar_sgd import CovarDataset,Covar,CovarTrainer,dataset_collate
cds = CovarDataset(sim,rank = rank)
covar = Covar(resolution=img_size,rank=rank,norm_factor=cds.im_norm_factor)#,vectors=vectorsGD)
batch_size = 32
learning_rate = 5e-4
momentum = 0.9
reg = 1e-5
gamma_lr = 0.8
gamma_reg = 1
device = torch.device('cuda:0')
dataloader = torch.utils.data.DataLoader(cds,batch_size = batch_size,shuffle = False)#,collate_fn=dataset_collate)
trainer = CovarTrainer(covar,dataloader,device,vectorsGD = vectorsGD/cds.im_norm_factor)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [11]:

from covar_distributed import trainParallel
import torch.multiprocessing as mp

world_size = torch.cuda.device_count()
kwargs_dict = {'max_epochs' : 10, 'lr' : learning_rate,'momentum' : momentum,'optim_type' : 'SGD','reg' : reg,'gamma_lr': gamma_lr,'gamma_reg' : gamma_reg}
trainParallel(covar,cds,batch_size = batch_size,vectorsGD = vectorsGD/cds.im_norm_factor,**kwargs_dict)



gpu : 5 ,cost value : 5.49e+04,  cosine sim : 0.04: 100%|██████████| 9/9 [00:04<00:00,  1.88it/s]
gpu : 4 ,cost value : 6.53e+04,  cosine sim : 0.04: 100%|██████████| 9/9 [00:04<00:00,  1.89it/s]
gpu : 6 ,cost value : 5.44e+04,  cosine sim : 0.04: 100%|██████████| 9/9 [00:04<00:00,  1.89it/s]
gpu : 3 ,cost value : 6.47e+04,  cosine sim : 0.04: 100%|██████████| 9/9 [00:04<00:00,  1.88it/s]
gpu : 2 ,cost value : 6.79e+04,  cosine sim : 0.04: 100%|██████████| 9/9 [00:04<00:00,  1.86it/s]
gpu : 1 ,cost value : 5.28e+04,  cosine sim : 0.04: 100%|██████████| 9/9 [00:04<00:00,  1.85it/s]
gpu : 0 ,cost value : 5.54e+04,  cosine sim : 0.04: 100%|██████████| 9/9 [00:04<00:00,  1.86it/s]
gpu : 0 ,cost value : 5.52e+04,  cosine sim : 0.46: 100%|██████████| 9/9 [00:04<00:00,  2.03it/s]
gpu : 5 ,cost value : 5.48e+04,  cosine sim : 0.46: 100%|██████████| 9/9 [00:04<00:00,  2.03it/s]
gpu : 6 ,cost value : 5.41e+04,  cosine sim : 0.46: 100%|██████████| 9/9 [00:04<00:00,  2.03it/s]
gpu : 4 ,cost value 

In [ ]:
trainer.train(10,learning_rate,momentum,'SGD',reg = reg)

In [ ]:
x = covar.state_dict()
x